In [1]:
from LIMxCMBL.init import *
from LIMxCMBL.kernels import *
import sys

# Lambda_idx = int(sys.argv[1])
# nbins = int(sys.argv[2])
# ell_idx = int(sys.argv[3])

Lambda_idx = 18
nbins = 100
ell_idx = 0

Lambda = Lambdas[Lambda_idx]

# CCAT-prime
zmin = 3.5
zmax = 8.1

oup_fname = '/scratch/users/delon/LIMxCMBL/I_auto/comb'
oup_fname += '_zmin_%.1f_zmax_%.1f_Lambda_idx_%d_nb_%d_l_%d_tplquad.npy'%(zmin, zmax, Lambda_idx, nbins,ell_idx)
print(oup_fname)

/scratch/users/delon/LIMxCMBL/I_auto/comb_zmin_3.5_zmax_8.1_Lambda_idx_18_nb_100_l_0_tplquad.npy


In [2]:
Omega_field = 8 * (np.pi/180)**2 #rad^2
Pei = 2.3e4 #Mpc^3 kJy^2 /sr^2 
chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))

In [3]:
from scipy.interpolate import interp1d, LinearNDInterpolator
from scipy.integrate import quad, quad_vec, trapezoid, tplquad

inner_dkparp_integral = np.load('/oak/stanford/orgs/kipac/users/delon/LIMxCMBL/inner_dkparp_integral.npy')
tmp_chibs = []
tmp_log_deltas = []
tmp_fnctn = []
for i in range(len(chibs)):
    for j in range(len(deltas)):
        tmp_chibs += [chibs[i]]
        tmp_log_deltas += [np.log10(deltas[j])]
        tmp_fnctn += [inner_dkparp_integral[ell_idx,i,j]]
        
f_inner_integral = LinearNDInterpolator(list(zip(tmp_chibs, tmp_log_deltas)), tmp_fnctn)
f_inner_integral_LoLo = interp1d(x = chibs, y = inner_dkparp_integral[ell_idx], axis = 0)

In [4]:
inner_dkparp_integral[ell_idx].shape

(256, 128)

In [5]:
chi_bin_edges = np.linspace(chimin*(1+1e-8), chimax*(1 - 1e-8), nbins + 1)
chi_bin_centers = (chi_bin_edges[1:] + chi_bin_edges[:-1])/2
dchi_binned = np.mean(np.diff(chi_bin_edges))

In [36]:
from tqdm import trange

# f_KLIM   = get_f_KI()
def f_cross_integrand(_chib, chip, chi):
    #I Lo
    _delta = np.abs(1 - chi/_chib) #(1)
    _delta = max(1e-6, min(_delta, 0.7))
    
    cross1_integrand = 0
    if(chimin <= 2*_chib - chi <= chimax and chimin <= chi <= chimax):
        cross1_integrand = 2 * f_KI1D(chi) #chi (1)
        cross1_integrand *= f_inner_integral((_chib, np.log10(_delta))) #(1)
        cross1_integrand /= (_chib**2) #(1)
        cross1_integrand *=  f_KILo(max(10, (2*_chib - chi)), external_chi = chip, Lambda=Lambda)

    #Lo I
    _delta = np.abs(1 - chip/_chib) #(1)
    _delta = max(1e-6, min(_delta, 0.7))
    
    cross2_integrand = 0
    if(chimin <= 2*_chib - chip <= chimax and chimin <= chip <= chimax):
        cross2_integrand = 2 * f_KI1D(chip) #chip (1)
        cross2_integrand *= f_inner_integral((_chib, np.log10(_delta))) #(1)
        cross2_integrand /= (_chib**2) #(1)
        cross2_integrand *=  f_KILo(max(10, (2*_chib - chip)), external_chi = chi, Lambda=Lambda)

    return cross1_integrand + cross2_integrand
    


In [45]:
from tqdm import trange

# f_KLIM   = get_f_KI()
def f_integrand(_chib, chip, chi):
    #I Lo
    _delta = np.abs(1 - chi/_chib) #(1)
    _delta = max(1e-6, min(_delta, 0.7))
    
    cross1_integrand = 0
    if(chimin <= 2*_chib - chi <= chimax and chimin <= chi <= chimax):
        cross1_integrand = 2 * f_KI1D(chi) #chi (1)
        cross1_integrand *= f_inner_integral((_chib, np.log10(_delta))) #(1)
        cross1_integrand /= (_chib**2) #(1)
        cross1_integrand *=  f_KILo(max(10, (2*_chib - chi)), external_chi = chip, Lambda=Lambda)

    #Lo I
    _delta = np.abs(1 - chip/_chib) #(1)
    _delta = max(1e-6, min(_delta, 0.7))
    
    cross2_integrand = 0
    if(chimin <= 2*_chib - chip <= chimax and chimin <= chip <= chimax):
        cross2_integrand = 2 * f_KI1D(chip) #chip (1)
        cross2_integrand *= f_inner_integral((_chib, np.log10(_delta))) #(1)
        cross2_integrand /= (_chib**2) #(1)
        cross2_integrand *=  f_KILo(max(10, (2*_chib - chip)), external_chi = chi, Lambda=Lambda)

    cross_integrand = cross1_integrand + cross2_integrand
    
    #LoLo
    plus = _chib*(1+deltas) #(d)
    mins = _chib*(1-deltas) #(d)
    _idxs = (chimin < plus) & (plus < chimax) & (chimin < mins) & (mins < chimax)
    LoLo_integrand  = np.where(_idxs,
                               f_KILo(plus, external_chi = chi, Lambda=Lambda) * f_KILo(mins, external_chi = chip, Lambda=Lambda),
                               0) #(d)
    LoLo_integrand += np.where(_idxs,
                               f_KILo(mins, external_chi = chi, Lambda=Lambda) * f_KILo(plus, external_chi = chip, Lambda=Lambda),
                               0) #(d)
    
    LoLo_integrand *= (2 / _chib) #(d)
    LoLo_integrand *= deltas
    LoLo_integrand *= f_inner_integral_LoLo(_chib)
    LoLo_integrand = trapezoid(x = np.log(deltas), y = LoLo_integrand, axis=-1)
    
    return LoLo_integrand - cross_integrand

In [46]:
f_integrand(8000, 8001, 8004)

-6.635861184860739e-09

In [8]:
def r():
    return np.random.uniform(chimin, chimax)

In [9]:
r()

7551.625483920592

In [10]:
_tmp = [ (r(), r(), r()) for _ in range(10000)]

In [11]:
import cProfile
cProfile.run('[f_integrand(a,b,c) for a,b,c in _tmp]')

         1379560 function calls in 2.862 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.908    0.000    2.852    0.000 4145848165.py:4(f_integrand)
        1    0.010    0.010    2.862    2.862 <string>:1(<listcomp>)
        1    0.000    0.000    2.862    2.862 <string>:1(<module>)
    10000    0.003    0.000    0.005    0.000 _base.py:1513(issparse)
    10000    0.220    0.000    0.300    0.000 _interpolate.py:690(_call_linear)
    10000    0.061    0.000    0.475    0.000 _interpolate.py:759(_evaluate)
    10000    0.050    0.000    0.112    0.000 _interpolate.py:774(_check_bounds)
    10000    0.004    0.000    0.030    0.000 _methods.py:47(_sum)
    20000    0.007    0.000    0.052    0.000 _methods.py:55(_any)
    10000    0.030    0.000    0.030    0.000 _methods.py:90(_clip)
    10000    0.015    0.000    0.595    0.000 _polyint.py:59(__call__)
    10000    0.010    0.000    0.090    0.000 _polyint.p

In [15]:
# i = 13
# li, ri = chi_bin_edges[i], chi_bin_edges[i+1]

# j = 14
# lj, rj = chi_bin_edges[j], chi_bin_edges[j+1]

# res, _ = tplquad(f_cross_integrand, li, ri, lj, rj, chimin, chimax,
#                 epsrel = 1e-3, epsabs = 0.0)

In [18]:
i = 13
li, ri = chi_bin_edges[i], chi_bin_edges[i+1]

j = 14
lj, rj = chi_bin_edges[j], chi_bin_edges[j+1]

res, _ = tplquad(f_integrand, li, ri, lj, rj, 10, chimax_sample,
                epsrel = 1e-3, epsabs = 0.0)

In [21]:
print(res/dchi_binned**2)

-8.460922465496914e-07


In [20]:
comb_binned = np.zeros((nbins, nbins))

In [ ]:
for i in range(nbins):
    print(i+1,'/',nbins)
    for j in trange(i, nbins):
        li, ri = chi_bin_edges[i], chi_bin_edges[i+1]
        lj, rj = chi_bin_edges[j], chi_bin_edges[j+1]
        comb_binned[i,j], _ = tplquad(f_integrand, li, ri, lj, rj, 10, chimax_sample,
                                      epsrel = 1e-3, epsabs = 0.0)
        comb_binned[i,j] /= dchi_binned**2
        comb_binned[j, i] = comb_binned[i,j]

In [ ]:
np.save(oup_fname, comb_binned)
print('outputted')